In [1]:
from opensearchpy import OpenSearch
from opensearch_dsl import Search
import pandas as pd

In [2]:
from streamlit_chat import config
NCP_CONFIG = config.NCP_CONFIG
EMB_CONFIG = config.HCX_EMBEDDING
OSH_CONFIG = config.OPENSEARCH
EMB_MODEL_NAME = "clir-sts-dolphin"

In [3]:
host = OSH_CONFIG['HOST']
port = 9200
auth = (OSH_CONFIG['ID'], OSH_CONFIG['PASSWD'])


In [6]:
client = OpenSearch(
    hosts=[{'host': host, 'port': port}],
    http_compress = True, # enables gzip compression for request bodies
    http_auth = auth,
    use_ssl = True,
    verify_certs = False,
    ssl_assert_hostname = False,
    ssl_show_warn = False,
    timeout = 300
)

In [7]:
index_name = "ncp_edu_sc2-emb-test-index"
index_body = {
  "settings": {
    "index": {
      "knn": True,
      "knn.algo_param.ef_search": 100
    }
  },
  "mappings": {
    "properties": {
        "origin-question":{
            "type": "text"
        },
        "origin-text":{
            "type": "text"
        },
        ("emb-model--" + EMB_MODEL_NAME): {
            "type": "text"
        },
        "vector": {
          "type": "knn_vector",
          "dimension": 1024,
          "method": {
            "name": "hnsw",
            "space_type": "cosinesimil",
            "engine": "nmslib",
            "parameters": {
              "ef_construction": 128,
              "m": 24
            }
          }
        }
    }
  }
}

In [8]:
response = client.indices.create(index=index_name, body=index_body)

In [9]:
from streamlit_chat.modules.HyperClovaX import HyperClovaX
hcx = HyperClovaX(clova_studio_api_key=EMB_CONFIG['X-NCP-CLOVASTUDIO-API-KEY'], apigw_api_key=EMB_CONFIG['X-NCP-APIGW-API-KEY'])
hcx.set_embedding(app_id=EMB_CONFIG['APP_ID'], model_name=EMB_MODEL_NAME)


In [10]:
df_csv = pd.read_csv(filepath_or_buffer='streamlit_chat/data/starcraft2.csv')

FileNotFoundError: [Errno 2] No such file or directory: '스타크래프트2_키워드_수정_cp1.csv'

In [208]:
origin_question = []
origin_text = []
embedding_text = []
# print(len(df['Completion']))
for idx, line in df_csv.iterrows():
    print(f"{idx + 1} / {len(df_csv['Completion'])}")
    res = hcx.embedding(text = line['Completion'])
    if res.status_code == 200:
        embedding_result = res.json()['result']['embedding']
        origin_question.append(line['Text'])
        origin_text.append(line['Completion'])
        embedding_text.append(embedding_result)
    else:
        print(f'error : {line["Completion"]}')

1 / 246
2 / 246
3 / 246
4 / 246
5 / 246
6 / 246
7 / 246
8 / 246
9 / 246
10 / 246
11 / 246
12 / 246
13 / 246
14 / 246
15 / 246
16 / 246
17 / 246
18 / 246
19 / 246
20 / 246
21 / 246
22 / 246
23 / 246
24 / 246
25 / 246
26 / 246
27 / 246
28 / 246
29 / 246
30 / 246
31 / 246
32 / 246
33 / 246
34 / 246
35 / 246
36 / 246
37 / 246
38 / 246
39 / 246
40 / 246
41 / 246
42 / 246
43 / 246
44 / 246
45 / 246
46 / 246
47 / 246
48 / 246
49 / 246
50 / 246
51 / 246
52 / 246
53 / 246
54 / 246
55 / 246
56 / 246
57 / 246
58 / 246
59 / 246
60 / 246
61 / 246
62 / 246
63 / 246
64 / 246
65 / 246
66 / 246
67 / 246
68 / 246
69 / 246
70 / 246
71 / 246
72 / 246
73 / 246
74 / 246
75 / 246
76 / 246
77 / 246
78 / 246
79 / 246
80 / 246
81 / 246
82 / 246
83 / 246
84 / 246
85 / 246
86 / 246
87 / 246
88 / 246
89 / 246
90 / 246
91 / 246
92 / 246
93 / 246
94 / 246
95 / 246
96 / 246
97 / 246
98 / 246
99 / 246
100 / 246
101 / 246
102 / 246
103 / 246
104 / 246
105 / 246
106 / 246
107 / 246
108 / 246
109 / 246
110 / 246
111 / 24

In [380]:
if len(origin_text) == len(embedding_text):
    dt_dict = {'origin-question': origin_question, 'origin_text': origin_text, 'embedding_text': embedding_text}

    df_final = pd.DataFrame(dt_dict)


In [381]:
len(df_final)

246

In [382]:
## bulk insert
sc2_datas = []
for idx, line in df_final.iterrows():
    # print(line['origin_text'])
    # index_no = '{ "index" : { "_index" : "' + index_name + '", "_id" : "' + str(idx + 1) + '" } }'
    # data = '{ "origin-text": "' + line['origin_text'] + '", "sc2-vector": ' + str(line['embedding_text']) + ' }'
    
    index_no = { "index" : { "_index" : index_name, "_id" : (idx + 1)} }
    data =  {"origin-question": line['origin-question'], "origin-text": line['origin_text'], "vector": line['embedding_text']}
    sc2_datas.append(index_no)
    sc2_datas.append(data)

In [383]:
client.bulk(sc2_datas)

{'took': 356,
 'errors': False,
 'items': [{'index': {'_index': 'ncp_edu_sc2-emb-test-index',
    '_id': '1',
    '_version': 1,
    'result': 'created',
    '_shards': {'total': 2, 'successful': 2, 'failed': 0},
    '_seq_no': 0,
    '_primary_term': 1,
    'status': 201}},
  {'index': {'_index': 'ncp_edu_sc2-emb-test-index',
    '_id': '2',
    '_version': 1,
    'result': 'created',
    '_shards': {'total': 2, 'successful': 2, 'failed': 0},
    '_seq_no': 1,
    '_primary_term': 1,
    'status': 201}},
  {'index': {'_index': 'ncp_edu_sc2-emb-test-index',
    '_id': '3',
    '_version': 1,
    'result': 'created',
    '_shards': {'total': 2, 'successful': 2, 'failed': 0},
    '_seq_no': 2,
    '_primary_term': 1,
    'status': 201}},
  {'index': {'_index': 'ncp_edu_sc2-emb-test-index',
    '_id': '4',
    '_version': 1,
    'result': 'created',
    '_shards': {'total': 2, 'successful': 2, 'failed': 0},
    '_seq_no': 3,
    '_primary_term': 1,
    'status': 201}},
  {'index': {'_inde

In [384]:
query_text = "불곰은 은폐가 가능한가요?"
query_text_emb = hcx.embedding(text=query_text)
query_text_emb = query_text_emb.json()['result']['embedding']

In [385]:
print(query_text_emb)

[0.60759, -0.67729545, -0.35742778, 0.58521575, -0.5121947, 0.07088789, 0.68653995, 0.24885032, 2.7283971, 1.005491, -0.32093754, -0.3704152, -1.1106602, -0.55746895, 0.36548254, 0.21439578, -0.37350348, 0.19370379, 0.2741777, -0.63142383, 0.50918037, -0.33156738, 0.03391445, -0.26645866, 0.8548748, 0.22116591, -0.7322652, -0.30248863, -1.1448774, 0.7519972, 0.6676084, -1.6731714, -0.1444686, -0.5656977, 0.05077041, -0.13933462, -0.18773851, 0.06263378, -0.8133707, -0.8139202, -0.16791937, -1.010959, 0.8724457, -0.44234744, 0.7131898, 0.020253431, -0.1255034, -0.24278645, 0.27819002, -0.8982232, -0.014185509, -0.12003167, 0.48782444, 1.0834659, 2.2550771, 0.10917423, 0.59514284, -0.4922039, -0.7515878, 0.123602755, 1.1539706, 1.308664, -0.50997204, -0.31244737, 0.27430943, 0.9641547, -0.27055305, -0.84765494, 1.9792111, 1.8378402, -0.10281262, -0.45901012, -0.1968953, -2.112547, -1.1279162, 0.085128896, 1.0806671, -0.744865, 1.582247, 0.39064592, -0.021807695, 0.12373624, -0.558584, 0.

In [386]:
## Text Search
search_body_text_t = {
  "size": 10,
  "query": {
    "match": {
      "origin_text": query_text
    }
  }
}
search_result_text_t = client.search(index=index_name, body=search_body_text_t)
print(len(search_result_text_t["hits"]["hits"]))
for s_text_t in search_result_text_t["hits"]["hits"]:
    print(s_text_t)
    print(f"{s_text_t['_score']} | {s_text_t['_source']['origin-text']}")

0


In [387]:
## Text Search
search_body_text_q = {
  "size": 10,
  "query": {
    "match": {
      "origin-question": query_text
    }
  }
}
search_result_text_q = client.search(index=index_name, body=search_body_text_q)
print(len(search_result_text_q["hits"]["hits"]))
for s_text_q in search_result_text_q["hits"]["hits"]:
    print(s_text_q)
    print(f"{s_text_q['_score']} | {s_text_q['_source']['origin-text']}")

0


In [388]:

## Normal KNN
search_body = {
  "size": 15,
  "query": {
    "knn": {
      "vector": {
        "vector": query_text_emb,
        "k": 15
      }
    }
  }
}
search_result = client.search(index=index_name, body=search_body)
print(len(search_result["hits"]["hits"]))
for s in search_result["hits"]["hits"]:
    print(f"{s['_score']} | {s['_source']['origin-text']}")

RequestError: RequestError(400, 'search_phase_execution_exception', "failed to create query: Field 'sc2-vector' is not knn_vector type.")

In [370]:
query_text = "바이킹의 공격력을 알려주세요"
query_text_emb = hcx.embedding(text=query_text)
query_text_emb = query_text_emb.json()['result']['embedding']

## Hybrid Search
search_body = {
  "_source": {
    "exclude": [
      "vector"
    ]
  },
  "query": {
    "hybrid": {
      "queries": [
        {
          "match": {
            "origin-question": {
              "query": query_text
            }
          }
        },
        {
          "match": {
            "origin_text": {
              "query": query_text
            }
          }
        },
        {
          "knn": {
            "vector": {
              "vector": query_text_emb,
              "k": 10
            }
          }
        }
      ]
    }
  },"size":10,
  "search_pipeline" : {
      "phase_results_processors": [
      {
        "normalization-processor": {
          "normalization": {
            "technique": "min_max"
          },
          "combination": {
            "technique": "arithmetic_mean",
            "parameters": {
              "weights": [
                0.2,
                0.1,
                0.7
              ]
            }
          }
        }
      }
    ]
  }
}
print(index_name)
# search_params = {
#     "search_pipeline" : "cw-nlp-search-pipeline"
# }
#search_result = client.search(index=index_name, body=search_body, params=search_params)

search_result = client.search(index=index_name, body=search_body)
print(len(search_result["hits"]["hits"]))
for s in search_result["hits"]["hits"]:
    print(f"{s['_score']} | {s['_source']['origin-text']}")

ncp_edu_sc2-emb-test-sss
10
0.75924915 | 바이킹은 높은 체력을 가진 기계 유닛으로 다양한 목적으로 사용 가능합니다.
- 생명력 : 135
- 방어력 : 0(+1)
- 특성 : 중장갑-기계
0.7 | 바이킹은 3가지 적용 가능한 업그레이드가 있습니다.
- 지능형 제어 장치 : 바이킹의 전투 모드를 빠르게 전환합니다.
- 우주선 무기 업그레이드 1, 2, 3 단계 : 바이킹의 공격력을 증가시킵니다.
- 차량 및 우주선 장갑 업그레이드 1, 2, 3 단계 : 바이킹의 방어력을 증가시킵니다.
0.6814124 | 바이킹은 두가지 기본 능력이 존재합니다.
- 돌격 모드 : 바이킹을 돌격 모드로 전환합니다. 지상으로 이동하며 지상 목표물만 공격합니다.
- 전투기 모드 : 바이킹을 전투기 모드로 전환합니다. 공중으로 이동하며 공중 목표물만 공격합니다.
0.66316986 | 바이킹의 공격 능력은 아래와 같습니다.

랜저 유도탄(Lanzer Torpedoes)을 사용하는 공중 공격 모드(전투기 모드)
- 공격력 : 10(+1) x 2 / 중장갑 상대 : 14(+1) x 2
- 사거리 : 9
- 공격 속도 : 2(아주빠름 1.43)
- 대상 : 공중 목표물 

개틀링 포(Gatling Cannon)을 사용하는 지상 공격 모드(돌격 모드)
- 공격력 : 12(+1) / 기계 상대 : 20(+2)
- 사거리 : 6
- 공격 속도 : 1(아주 빠름 기준 0.71)
- 대상 : 지상 목표물
0.5636949 | 바이킹(Viking)은 스타크래프트2에 등장하는 테란 공중 유닛 입니다. 공중을 공격할 수 있는 전투기 모드와, 지상에 내려와 지상 유닛을 공격할 수 있는 돌격 모드 전환이 가능합니다.
0.34402 | 스타크래프트2에 등장하는 테란 공중유닛 정보를 알려드리겠습니다.
- 바이킹(Viking) : 튼튼한 전투 지원기 입니다. 강력한 대주력함 공중 미사일을 탑재하고 있습니다. 돌격 모드로 변신하여 지상에 내려와 지상 유닛을 공격할 수 있습니다.
-